In [27]:
import pandas as pd
from ast import literal_eval
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform
from os.path import join as os_join
import seaborn as sns
import matplotlib.pylab as plt
from pyvis import network as net


pd.set_option('display.max_colwidth', None)

In [28]:
similarity_path = "../data/processed/vis_dataset/similarity_combined.csv"
dataset_path = "../data/processed/vis_dataset/vis_data.csv"

output_path = "html_files/reference_similarity.html"

In [29]:
dataset = pd.read_csv(dataset_path)

dataset["Title"] = dataset["Title"].astype(str)
dataset["AuthorNames"] = dataset["AuthorNames"].astype(str)


dataset.sample(2)

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
140,InfoVis,2003,Design choices when architecting visualizations,10.1109/INFVIS.2003.1249007,140,"[108, 112, 21, 60, 127]","[269, 323]",http://dx.doi.org/10.1109/INFVIS.2003.1249007,41,48,...,"In this paper, we focus on some of the key design decisions we faced during the process of architecting a visualization system and present some possible choices, with their associated advantages and disadvantages. We frame this discussion within the context of Rivet, our general visualization environment designed for rapidly prototyping interactive, exploratory visualization tools for analysis. As we designed increasingly sophisticated visualizations, we needed to refine Rivet in order to be able to create these richer displays for larger and more complex data sets. The design decisions we discuss in this paper include: the internal data model, data access, semantic meta-data information the visualization can use to create effective visual decodings, the need for data transformations in a visualization tool, modular objects for flexibility, and the tradeoff between simplicity and expressiveness when providing methods for creating visualizations.",Diane Tang;Chris Stolte;Robert Bosch,D. Tang;C. Stolte;R. Bosche,"Stanford Univ., CA, USA;Stanford Univ., CA, USA;Stanford Univ., CA, USA",10.1109/INFVIS.1996.559213;10.1109/INFVIS.2002.1173156;10.1109/VISUAL.1995.480801;10.1109/INFVIS.1998.729560;10.1109/INFVIS.2002.1173141;10.1109/INFVIS.2000.885086;10.1109/VISUAL.1992.235219,"information visualization, system architecture, semantic meta-data, data transformations, design tradeoffs",13.0,4.0,30.0,NaN
1091,VAST,2014,Using Visualizations to Monitor Changes and Harvest Insights from a Global-Scale Logging Infrastructure at Twitter,10.1109/VAST.2014.7042487,1091,"[353, 34, 835, 964, 37, 641, 458, 106, 940, 208, 311, 285, 767]",[1319],http://dx.doi.org/10.1109/VAST.2014.7042487,113,122,...,"Logging user activities is essential to data analysis for internet products and services. Twitter has built a unified logging infrastructure that captures user activities across all clients it owns, making it one of the largest datasets in the organization. This paper describes challenges and opportunities in applying information visualization to log analysis at this massive scale, and shows how various visualization techniques can be adapted to help data scientists extract insights. In particular, we focus on two scenarios: (1) monitoring and exploring a large collection of log events, and (2) performing visual funnel analysis on log data with tens of thousands of event types. Two interactive visualizations were developed for these purposes: we discuss design choices and the implementation of these systems, along with case studies of how they are being used in day-to-day operations at Twitter.",Krist Wongsuphasawat;Jimmy Lin,Krist Wongsuphasawat;Jimmy Lin,"Twitter, Inc.;Twitter, Inc.",10.1109/INFVIS.2000.885091;10.1109/TVCG.2009.117;10.1109/INFVIS.1997.636718;10.1109/VAST.2007.4389008;10.1109/INFVIS.1996.559227;10.1109/TVCG.2012.225;10.1109/TVCG.2007.70529;10.1109/VAST.2012.6400494;10.1109/TVCG.2013.231;10.1109/INFVIS.2004.64;10.1109/VISUAL.1991.175815;10.1109/TVCG.2013.200;10.1109/VAST.2006.261421;10.1109/TVCG.2011.185,"Information Visualization, Visual Analytics, Log Analysis, Log Visualization, Session Analysis, Funnel Analysis",11.0,13.0,45.0,NaN


In [30]:
id_title_abstract = "<b>Paper id</b>: " + dataset["id"].astype(str) + " ("  + dataset["Conference"] + ") " " - "\
"<a href=\""  + dataset.Link + "\" target=\"_blank\">Link</a>" + \
"; <br><b>Authors: </b>: " + dataset["AuthorNames"] + \
"; <br><b>Title</b>: " + dataset["Title"] + \
"; <br><b>Abstract</b>: " + dataset["Abstract"]

map_id_to_label={k: v for  k,v in enumerate(id_title_abstract)}
map_id_to_label[0]

'<b>Paper id</b>: 0 (InfoVis)  - <a href="http://dx.doi.org/10.1109/INFVIS.1995.528680" target="_blank">Link</a>; <br><b>Authors: </b>: B. Spence;L. Tweedie;H. Dawkes;Hua Su; <br><b>Title</b>: Visualisation for functional design; <br><b>Abstract</b>: We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken o

In [31]:
df = pd.read_csv(similarity_path)
df

,id,0,1,2,3,4,5,6,7,8,...,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1742,1742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1743,1743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1744,1744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
arr = df.values
index_names = df.index
col_names = df.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>0.0)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=['from','to','value'])
df_out["from"] = df_out["from"].astype(np.int16)
df_out["to"] = df_out["to"].astype(np.int16)
df_out

,from,to,value
0,0,238,1.0
1,0,244,1.0
2,0,456,1.0
3,0,512,1.0
4,0,550,1.0
...,...,...,...
60213,1611,1612,2.0
60214,1611,1614,1.0
60215,1611,1615,2.0
60216,1614,1615,1.0


## Plot using Pyvis

In [33]:
color_dict = {
    "InfoVis": "blue",
    "VAST": "orange",
    "SciVis": "red"
}

def get_node_color(node, df, color_dict):
    conference = df[df.id == node].Conference.values[0]
    return color_dict[conference]

In [39]:
thres = 4
df_out_thres = df_out[df_out["value"] >= thres]
plotting_all_nodes = False

file_name= output_path
# g = net.Network(height='95%', width='99%',heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')

g = net.Network(height="500px", width="100%",\
                heading='Paper Similarity using References')
g.set_template("html_files/template_2.html")
g.set_edge_smooth('discrete')
# g.set_options('{"nodes": {"borderWidthSelected": "10" } }')

if plotting_all_nodes:
    node_list = list(df.index.astype(np.int16))
else:
    node_list = list(set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) )

node_labels = [ map_id_to_label.get(node) for node in node_list]
node_colors = [ get_node_color(node, dataset, color_dict) for node in node_list]
    
# g.add_nodes(node_list, title=node_labels, label=node_list, color=node_colors)

g.add_nodes(node_list, title=node_labels, color=node_colors)


edge_list = zip(df_out_thres["from"], df_out_thres["to"], df_out_thres['value'])
g.add_edges(edge_list)    
# g.show_buttons(filter_=["physics"])
# g.show_buttons()




g.set_options("""
var options =   {
        "physics": { "stabilization": false ,
            "barnesHut" : {"centralGravity":0.1,     "gravitationalConstant":-5000}
                    },
 "edges": { "smooth": { "type": "continuous" } } 
 
}
 
  """)

g.show(file_name)

#  source code: https://github.com/WestHealth/pyvis/blob/523d541d04df6779dd21e5942af60b79369e8f13/pyvis/network.py#L218

In [40]:
!open $file_name

In [73]:
g.num_nodes()

628

In [41]:
import dill

#### Write
with open('/Users/chaupham/minhchau/MY_WORK/C/citation_networks/src/streamlit_network/models/reference_similarity.pkl', 'wb') as f:     
    dill.dump(g, f)

In [42]:
file_name

'html_files/reference_similarity.html'

In [43]:
!cp /Users/chaupham/minhchau/MY_WORK/C/citation_networks/src/streamlit_network/models/reference_similarity.pkl /Users/chaupham/Downloads/streamlit_network_2/models/
!cp $file_name /Users/chaupham/Downloads/streamlit_network_2/html_files/



In [ ]:
!open .

## Plot using Jaal

More elegant and faster, but still in its infancy though. There's a lack of some crucial features such as "edge thickness"

In [ ]:
dataset

In [ ]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got


thres = 6
plotting_all_nodes = False


    
    

df_out_thres = df_out[df_out["value"] >= thres]
edges = df_out_thres

(dataset)
# edges.loc[:, 'label'] = edges.loc[:, 'value'].astype(int).astype(str)

edges["from"] = edges["from"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0]))
edges["to"] = edges["to"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 


set_thres_nodes = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) )
nodes = pd.DataFrame({"id": df.index})
nodes["having_edge"] = nodes["id"].map(lambda x: x in set_thres_nodes).astype(str)
nodes["conference"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Conference"].values[0])
nodes["Title"] = nodes["id"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 
nodes["id"] = nodes["Title"]
if plotting_all_nodes:
    pass
else:
    print(nodes.shape)
    node_list = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) 
    nodes = nodes[nodes["id"].isin(node_list)]
    print(nodes.shape)

nodes.to_csv("../data/interim/jaal_data/nodes_2.csv", index=False)
edges.to_csv("../data/interim/jaal_data/edges_2.csv", index=False)

port=8050

while True:
    try:
        Jaal(edges, nodes).plot(port=port)
    except:
        port+=1


In [ ]:
edges["value"] = edges["value"].astype(int)

In [ ]:
edges.nlargest(10,"value")

In [ ]:
node_list

In [ ]:
nodes[nodes.id==1606].Title.values[0]

In [ ]:
edges[(edges["to"] == 755) & (edges["from"]==456)]

In [ ]:
pip install networkx

In [ ]:
import networkx as nx
from pyvis.network import Network


nx_graph = nx.cycle_graph(10)
nx_graph.nodes[1]['title'] = 'Number 1'
nx_graph.nodes[1]['group'] = 1
nx_graph.nodes[3]['title'] = 'I belong to a different group!'
nx_graph.nodes[3]['group'] = 10
nx_graph.add_node(20, size=20, title='couple', group=2)
nx_graph.add_node(21, size=15, title='couple', group=2)
nx_graph.add_edge(20, 21, weight=5)
nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)

nt = Network(notebook=True, height="750px", width="100%")

nt.from_nx(nx_graph)
nt.show("nx.html")